In [1]:
import pandas as pd

In [2]:
import pywikibot as pb
site = pb.Site("wikidata", "wikidata")
repo = site.data_repository()

In [3]:
from bs4 import BeautifulSoup
import requests
import dateparser

In [ ]:
!pip install xlrd

In [ ]:
!pip install XlsxWriter

In [ ]:
def create_item(site, label_dict):
    new_item = pb.ItemPage(site)
    new_item.editLabels(labels=label_dict, summary="Creating Red List item")
    return new_item

In [ ]:
def set_claim(item, property, string_data):
    clm = new_item.get()['claims'][property][0]
    clm_trgt = clm.getTarget()
    print(clm_trgt)
    if clm_trgt != string_data :
        claim = pb.Claim(repo, property)
        claim.setTarget(string_data)
        item.addClaim(claim, bot=False, summary="Test")
        return claim
    else :
        return None

In [8]:
red_list_df = pd.read_excel("red2.xlsx")

In [9]:
red_list_df.columns

Index(['Número', 'Nombre', 'Artículo en español', 'Wikidata ID', 'Latitud',
       'Longitud', 'Localidad', 'Localidad ID', 'Municipio', 'Municipio ID',
       'Provincia', 'Comunidad autónoma', 'Tipo', 'Tipo 2', 'Fecha inclusión',
       'Fecha salida', 'Estatus retirada', 'Descripción', 'URL', 'Estatus',
       'Categoría', 'Imagen'],
      dtype='object')

In [10]:
output_df = pd.DataFrame(columns=red_list_df.columns)
for index, row in red_list_df.iterrows():
    if pd.notnull(row["Wikidata ID"]) :
        print ("--------------")
        item = pb.ItemPage(repo, row["Wikidata ID"])
        item.get()
        commons_cat = None
        if item.sitelinks and "commonswiki" in item.sitelinks and "Category:" in item.sitelinks["commonswiki"]:
            print (item.sitelinks["commonswiki"])
            commons_cat = item.sitelinks["commonswiki"].split(':')[1].strip()
            
        if item.claims and 'P373' in item.claims: # instance of
            print(item.claims['P373'][0].getTarget())
            commons_cat = item.claims['P373'][0].getTarget()

        if commons_cat != None :
            row["Categoria"] = commons_cat

    output_df = output_df.append(row, ignore_index=True)        

--------------
Acueducto de Albatana
--------------
--------------
Category:Alcázar de Toro
Alcázar de Toro
--------------
--------------
--------------
--------------
Alqueria del Moro
--------------
Alqueria del Pi d'Alfafar
--------------
Category:Palacio de Chiloeches, Santoña
Palacio de Chiloeches, Santoña
--------------
Category:Church of San Miguel, Sasamón
Church of San Miguel, Sasamón
--------------
--------------
--------------
Category:Tower of Valmojado
Tower of Valmojado
--------------
Category:Bahía de Santander
Bahía de Santander
--------------
--------------
Category:Cabanyal-Canyamelar
Cabanyal-Canyamelar
--------------
Category:Batería de Cenizas
Batería de Cenizas
--------------
--------------
--------------
--------------
Category:Capilla de San José, Sevilla
Capilla de San José (Sevilla)
--------------
Cartuja de Nuestra Señora de las Fuentes
--------------
--------------
Cartuja de Vall de Cristo
--------------
--------------
--------------
--------------
Category

Category:Hermitage of Virgen de la Oliva, Patones
Hermitage of Virgen de la Oliva, Patones
--------------
Hermitage of Gañarul
--------------
Category:Hermitage of Nuestra Señora de la Calzada, Brías
Hermitage of Nuestra Señora de la Calzada, Brías
--------------
Category:Ermita de San Ambrosio (Barbate)
Ermita de San Ambrosio (Barbate)
--------------
Category:Hermitage of San Juan de Socueva
Hermitage of San Juan de Socueva
--------------
--------------
Category:San Pedro de la Mata, Casalgordo
San Pedro de la Mata, Casalgordo
--------------
Category:Hermitage of San Pelayo, Perazancas de Ojeda
Hermitage of San Pelayo, Perazancas de Ojeda
--------------
--------------
Category:Ermita de Santa Ana, El Toboso
Ermita de Santa Ana, El Toboso
--------------
--------------
Ermita del Calvari de Canet lo Roig
--------------
Category:Ermita del Santo Cristo (Talaván)
Ermita del Santo Cristo (Talaván)
--------------
Hermitage of San Miguel (Sacramenia)
--------------
Church of San Miguel Árcan

Palacio de Nava, San Cristóbal de La Laguna
--------------
--------------
Category:Palace of Tormaleo
Palace of Tormaleo
--------------
--------------
Category:Palace of Valsaín
Palace of Valsaín
--------------
--------------
--------------
Category:Conde Luna Palace
Conde Luna Palace
--------------
Category:Palacio del Infantado o del Condestable, Colindres
Palacio del Infantado o del Condestable, Colindres
--------------
Marqués de Revilla's Palace, Navares de las Cuevas
--------------
Guenduláin
--------------
Category:Mosteiro de San Paio de Diomondi
Mosteiro de San Paio de Diomondi
--------------
Category:Palacio de los Zúñiga (Curiel de Duero)
Palacio de los Zúñiga (Curiel de Duero)
--------------
--------------
Doñana National Park
--------------
Category:Peñalcazar
Peñalcazar
--------------
Mosteiro de San Xulián de Moraime
--------------
--------------
Old town hall of Laredo
--------------
Category:Alcántara Bridge
Alcántara Bridge
--------------
--------------
Category:Puent

In [ ]:
top_categories = ['Patrimonio industrial', 
                  'Patrimonio Religioso', 
                  'Patrimonio Militar', 
                  'Patrimonio Civil', 
                  'Patrimonio Natural', 
                  'Patrimonio Arqueológico']
output_df = pd.DataFrame(columns=red_list_df.columns)
for index, row in red_list_df.iterrows():
    print ("-----------------------------")
    url = row['URL']
    text = requests.get(url).text
    table = BeautifulSoup(text, 'html.parser').find("div", {"class": "post-meta"})
    typology = [i.text for i in table.find_all('li') if "Tipología" in i.text][0]
    categories = [i.strip() for i in typology.replace("Tipología: ", "").split(',')]
    type1_categories = [cat for cat in categories if cat in top_categories and cat != 'Retirados de la lista']
    type2_categories = [cat for cat in categories if cat not in top_categories and cat != 'Retirados de la lista']
    row['Tipo 2'] = '|'.join(type2_categories)
    row['Tipo'] = '|'.join(type1_categories)
    
    start = [i.text for i in table.find_all('li') if "inclusión" in i.text][0]
    start_date = dateparser.parse(start.replace("Fecha de inclusión: ", "").replace(", ", " de "))
    print (start_date.strftime("%Y-%m-%d"))
    row["Fecha inclusión"] = start_date.strftime("%Y-%m-%d")

    try:
        end = [i.text for i in table.find_all('li') if "retirada" in i.text][0]
        end_date = dateparser.parse(end.replace("Fecha de retirada: ", "").replace(", ", " de "))
        print (end_date.strftime("%Y-%m-%d"))
        row["Fecha salida"] = end_date.strftime("%Y-%m-%d")
    except :
        pass

    output_df = output_df.append(row, ignore_index=True)

In [11]:
writer = pd.ExcelWriter('simple.xlsx', engine='xlsxwriter')
output_df.to_excel(writer, sheet_name='Sheet1')
writer.save()